## Libraries and Utilities

In [1]:
# Generic python libraries
import os
import math
import random
from PIL import Image
from random import sample

# PyTorch and numpy related
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split

# External tool (built by us) to format the dataset structure
if not os.path.exists('utils.py'):
  !wget https://raw.githubusercontent.com/humbertordrgs/VR_DL_2/master/utils/utils.py

# External dataset class (built by us)
if not os.path.exists('sbir_dataset.py'):
  !wget https://raw.githubusercontent.com/humbertordrgs/VR_DL_2/master/utils/sbir_dataset.py

from utils import load_images, get_class_map, get_processed_img, execution_time
from sbir_dataset import SketchBasedImageRetrievalDataset

--2021-05-23 05:50:25--  https://raw.githubusercontent.com/humbertordrgs/VR_DL_2/master/utils/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1385 (1.4K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   1.35K  --.-KB/s    in 0s      

2021-05-23 05:50:26 (27.7 MB/s) - ‘utils.py’ saved [1385/1385]

--2021-05-23 05:50:26--  https://raw.githubusercontent.com/humbertordrgs/VR_DL_2/master/utils/sbir_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2501 (2.4K) [text/plain]
Saving to

## Dowloading Datasets

In [2]:
# Sketches dataset
if not os.path.exists('Sketch_EITZ.zip'):
  !wget https://www.dropbox.com/s/ut350iwgby9swk2/Sketch_EITZ.zip && unzip -q Sketch_EITZ.zip

# Gallery Image Retrieval for Train and validation
if not os.path.exists('Flickr25K.zip'):
  !wget https://www.dropbox.com/s/khbxruh3acq84eg/Flickr25K.zip && unzip -q Flickr25K.zip

# Gallery Image Retrieval for Test
# if not os.path.exists('Flickr15K.zip'):
#   !wget https://www.dropbox.com/s/q5ew09x4e3rsiht/Flickr15K.zip && unzip -q Flickr15K.zip


--2021-05-23 05:50:30--  https://www.dropbox.com/s/ut350iwgby9swk2/Sketch_EITZ.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ut350iwgby9swk2/Sketch_EITZ.zip [following]
--2021-05-23 05:50:30--  https://www.dropbox.com/s/raw/ut350iwgby9swk2/Sketch_EITZ.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc79f1e6ba92cbac52a9042aadd0.dl.dropboxusercontent.com/cd/0/inline/BPCUtNSrIGdT3rUC0NbQMIjJ31zxhqsWuU-71KJEW9aRbyPQN5ErAu44M3r8d5txNSjJn1g0r1E-7C3M1MfBMffJtehLliMjedfigQ_dSuEWtyqOq81hg9z_Ri_sU3EaaIrOgw4wO02b3CEe2MOQvf7X/file# [following]
--2021-05-23 05:50:30--  https://uc79f1e6ba92cbac52a9042aadd0.dl.dropboxusercontent.com/cd/0/inline/BPCUtNSrIGdT3rUC0NbQMIjJ31zxhqsWuU-71KJEW9aRbyPQN5ErAu44M3r8d5txNSjJn1g0r1E-7C3M1

In [3]:
train_dataset = SketchBasedImageRetrievalDataset("Sketch_EITZ", "train.txt", "Flickr25K", "mapping.txt", use_triplets=True)

Constructor: 89.41279 seconds


In [4]:
torch.save(train_dataset, 'SBIR_train_dataset.pt')

In [5]:
!zip SBIR_train_dataset.zip SBIR_train_dataset.pt

  adding: SBIR_train_dataset.pt (deflated 84%)


In [9]:
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
!cd Dropbox-Uploader
!chmod +x dropbox_uploader.sh
!echo "pYnsrTWgA48AAAAAAAAAARX2U63Rm1pdHoAByjVChNPoOPc8ENnqAagihgzecqoF" > token.txt

fatal: destination path 'Dropbox-Uploader' already exists and is not an empty directory.

 This is the first time you run this script, please follow the instructions:

 1) Open the following URL in your Browser, and log in using your account: https://www.dropbox.com/developers/apps
 2) Click on "Create App", then select "Dropbox API app"
 3) Now go on with the configuration, choosing the app permissions and access restrictions to your DropBox folder
 4) Enter the "App Name" that you prefer (e.g. MyUploader126092337121797)

 Now, click on the "Create App" button.

 When your new App is successfully created, please click on the Generate button
 under the 'Generated access token' section, then execute the following command:

 echo "INPUT_YOUR_ACCESS_TOKEN_HERE" > token.txt


 Found token.txt with access token: pYnsrTWgA48AAAAAAAAAARX2U63Rm1pdHoAByjVChNPoOPc8ENnqAagihgzecqoF

 You can unlink this account if the token is wrong, using:
 ./dropbox-uploader.sh unlink

 The configuration has be

In [10]:
!bash dropbox_uploader.sh upload ../SBIR_train_dataset.zip /

 > Uploading "/content/SBIR_train_dataset2.zip" to "/SBIR_train_dataset2.zip" by 44 chunks ............................................ DONE
